In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_29810/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 07:31:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 07:31:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
# Sample DataFrame (replace this with your actual DataFrame)
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

df_person = spark.createDataFrame(employee)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Employee")

df_person = spark.createDataFrame(department)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Department")



In [8]:
query = """
	select
		Department,
		Employee,
		Salary
	from (select
		e.id as employee_id,
		e.name as Employee,
		e.salary as Salary,
		d.name as Department,
		dense_rank() over (partition by d.name order by e.salary desc) as rank_salary
	from
		Employee e left join Department d on e.departmentId = d.id)
	where rank_salary = 1
	

"""

In [9]:
spark.sql(query).show()

+----------+--------+------+
|Department|Employee|Salary|
+----------+--------+------+
|        IT|     Jim| 90000|
|        IT|     Max| 90000|
|     Sales|   Henry| 80000|
+----------+--------+------+

